# ECE 364 Lecture 22
## Deep Nets (Part 4): Convolutional Neural Networks History and Recurrent Neural Networks
### Learning Objectives
After this lecture, students will be able to
* List key developments in the use of convolutional neural networks for image classification.
* Explain how recurrent neural networks (RNN) and convolutional neural networks differ.
* Identify the equations that represent a long short-term memory (LSTM) RNN and use PyTorch to create an LSTM RNN.

## Convolutional Neural Networks History
### Image Classification
The first task to greatly benefit from the use of convolutional neural networks (CNN) was image classification. We have already seen image classification multiple times in this course. In brief, image classification takes an input image and seeks to assign one of a finite collection of labels to the image. For example, we may seek to classify images of animals as being one of: cat, dog, horse, frog, giraffe, or cow for a 6-class classification problem. The below timeline provides an incomplete set of events that greatly advanced the use of CNNs and deep learning in general.
#### LeNet (1998)
The first major practical success was the [LeNet](https://en.wikipedia.org/wiki/LeNet) (specifically LeNet-5) model developed by Yann LeCun, et al. at Bell Laboratories. LeNet was developed for handwritten digit classification on the MNIST dataset and helped automate the processing of checks and mail zip codes.
<div>
    <center><img src="LeNet-5.png" width="200"</center>
</div>

#### ImageNet Dataset (2009-)
The [ImageNet Dataset](https://en.wikipedia.org/wiki/ImageNet), developed by Fei-Fei Li, et al., is credited with accelerating the development of image classification models and consequently CNNs. The dataset contains multiple different tasks and has grown in size since its introduction in 2009. The most famous application of ImageNet was for the "ImageNet Large Scale Visual Recognition Challenge" (ILSVRC). While this challenge has evolved over the years, the most popularly cited version is ImageNet1k which refers to the challenge with 1,000 classes across 1,281,167 images. Example images may be easily viewed in [this GitHub repository](https://github.com/EliSchwartz/imagenet-sample-images/tree/master).

#### AlexNet (2012)
Prior to 2012, CNNs had not become too popular in the computer vision and machine learning community in spite of the success of LeNet back in 1998. Limitations on compute and specialized hardware posed a serious roadblock for developing CNNs and as such, more classical models like SVMs remained the dominant methods in the machine learning community. In 2012, [AlexNet](https://en.wikipedia.org/wiki/AlexNet) became the first CNN to win the ILSVRC competition at the CVPR 2012 conference with a top-5 error rate of 15.3%, more than 10.8% better than the runner-up. Developed by Alex Krizhevsky, Ilya Sutskever (co-founder of OpenAI), and Geoffrey Hinton (recent Nobel Prize winner), AlexNet represented a large shift in attention for researchers towards deep learning for computer vision. 

The key contributions of AlexNet were the use of deeper and larger CNNs (eight layers), ReLU non-linearity, and graphics processing units (GPU) to help with the computational burden of training such a large model on a similarly large dataset. The below figure depicts the model architectures of LeNet and AlexNet side-by-side.

<div>
    <center><img src="AlexNet.png" width="400"</center>
</div>


#### VGG (2014)
Following the success of AlexNet, there was increased interest in developing deeper CNN models. However, these deeper models are of course exceptionally expensive to train. The VGG-16 and VGG-19 models developed by Karen Simonyan and Andrew Zisserman of the Visual Geometry Group at University of Oxford gave a quite simple technique for effectively training deeper CNNs: just use smaller kernels! The VGG-16 and VGG-19 models only use $3\times 3$ convolutional kernels where previous models often made use of multiple larger kernel sizes.

The downside of smaller kernels is that they see less of the input image or feature map and thus have a narrower **receptive field** on their own. However, Simonyan and Zisserman showed that multiple $3\times 3$ kernel layers may actually achieve the same receptive field size as a larger kernel size with many fewer parameters. For example, consider a single convolutional layer with a $7\times 7$ kernel. For simplicity, also assume the number of input and output channels $C$ is kept the same. Thus, this layer requires $7\times 7\times C\times C=49C^2$ learnable parameters. Instead, we may stack three successive convolutional layers each with $3\times 3$ kernels (and activation functions in between). The first layer has a receptive field size of $3\times 3$. The second layer increases this receptive field to $5\times 5$, while the third layer reaches the $7\times 7$ size of the larger-kernel layer. Each of these smaller convolutional layers only requires $3\times 3\times C\times C=9C^2$ paremeters. Therefore, the entire stack of three layers takes only $27C^2$ parameters!

This simple but effective insight allows us to build deeper models with a lower burden on fitting parameters while maintaining the same ability to build feature representations at growing scales through the image. The below tables are taken from the [original VGG paper](https://arxiv.org/pdf/1409.1556).

<div>
    <center><img src="vgg.png" width="600"</center>
</div>

#### ResNet (2015)
As researchers made CNN models deeper, a new phenomenon was observed. While depth was helpful, there reached a point where performance would not only saturate but actual get *worse*! The below figure is taken from the original ResNet paper and demonstrates this.
<div>
    <center><img src="deeper-error.png" width="400"</center>
</div>

This phenomenon is counter-intuitive since an $N+L$-layer model should at least as good as an $N$-layer model. A simple thought experiment is to imagine the added $L$ layers as all performing an identity mapping, thus making the deeper and shallower models behave the same. In 2015, He, et al. argued that deeper models struggle to learn this potential identity mapping where performance begins to saturate and thus additional layers may not be necessary. This inspired the creation of the **residual unit** and the resulting ResNet model build of these residual units. Mathematically, let $\mathcal{F}(x)$ represent some function applied to input $x$. The authors claim that in deeper layers, it is quite difficult for the model (starting from some random initialization or learned set of values) to converge to an identity mapping for such an $\mathcal{F}(x)$. Instead, the authors propose the residual unit $\mathcal{H}(x)=\mathcal{F}(x)+x$ where a **skip connection** combines the input to the unit with the result of the learnable weight layers of the unit applied to that input. In this way, an identity mapping may be learned by $\mathcal{F}(x)=0$, which is easier to optimize than the very particular identity mapping. The below figure from the ResNet paper shows this residual block.
<div>
    <center><img src="residual-block.png" width="400"</center>
</div>

The use of residual blocks allowed for dramatically deeper models to be trained, e.g. ResNet-34 shown below from the original paper, such at ResNet-50 and ResNet-101 at the time. Today, residual blocks are at the heart of numerous successful deep learning models and are critical to the ability to train exceptional deep and large models.

<div>
    <center><img src="resnet-34.png" width="400"</center>
</div>

## Recurrent Neural Networks
The deep net architectures we have explored thus far, both MLPs and CNNs, are referred to as **feed-forward** neural networks since inputs are passed to a deep net and the computation simply proceeds by a single forward pass through the model. Thus, the model has no memory or state from the previous input when receiving the next input. This may be undesirable when processing sequence data wherein the result at one time step clearly depends on previous time steps. As such, we would like a neural network architecture that has some notion of memory/state as processing from one time step may influence processing.

Let $x\in\mathbb{R}^{T\times d}$ be an example of sequence data where $x=\{x_0, x_1, x_2, \ldots, x_{T-1}\}$ and $x_t\in\mathbb{R}^d$. For each time step, our model $f$ takes input $x_t$ and the **hidden state** of computation from previous time step $t-1$, $h_{t-1}\in\mathbb{R}^{H}$. Thus,
$$
f(x_t, h_{t-1}) = h_t.
$$

Such a model is referred to as a **recurrent neural network** (RNN) since the model's computation forms a recurrence, i.e. output at each time depends on outputs at previous times.

For whatever task of interest, we have a hidden state of the model at each time step that is an $H$-dimensional vector. We may then apply a fully-connected layer to these each hidden state vector for an appropriate regression or classification problem. Consider, for example, a collection of sensors at a traffic intersection. At each time step, we can have several sensor readings, e.g. in-roadway induction loop, radar sensors, data from cameras, etc. An RNN will produce some **latent representation** $h_t\in\mathbb{R}^H$ at each step that we may use to estimate the speed of a vehicle (regression problem) or classify what kind of vehicle is passing through the intersection (classification problem).

### Long Short-Term Memory Networks
There has been much development over the years of various fundamental layers/units that form the $f(x_t, h_{t-1})$ shown above for RNNs. The most popular such architecture is the **long short-term memory** (LSTM) RNN. There are many pieces of computation that constitute one LSTM layer or **cell**: hidden state, cell state, input gate, forget gate, cell gate, and output gate.

At time step $t$, the LSTM has input $x_t$, previous hidden state $h_{t-1}$, and cell state at previous time step $c_{t-1}$. The LSTM cell computes the following non-linear activation values:
$$
\begin{align}
    i_t &= \sigma(W_{ii}x_t +W_{hi}h_{t-1})\in[0, 1],\quad&\textrm{Input Gate Activation}\\
    f_t &= \sigma(W_{if}x_t+W_{hf}h_{t-1})\in[0, 1],\quad&\textrm{Forget Gate Activation}\\
    g_t &= \textrm{tanh}(W_{ig}x_t+W_{hg}h_{t-1})\in[-1, 1],\quad&\textrm{Cell Gate Activation}\\
    o_t &= \sigma(W_{io}x_t + W_{ho}h_{t-1})\in[0, 1],\quad&\textrm{Output Gate Activation}
\end{align}
$$
Above, $\sigma$ denotes the sigmoid function and $\textrm{tanh}$ is the hyperbolic tangent function. The matrices $W_{ii}, W_{hi}, W_{if}, W_{hf}, W_{ig}, W_{hg}, W_{io}, W_{ho}$ represent the learnable weights in the LSTM cell. These respective gate activations control how the cell state and hidden state at the current time-step are updated. We obtain the present cell state and hidden state via:

$$
\begin{align}
    c_t &= f_t\odot c_{t-1}+i_t\odot g_t\\
    h_t &= o_t\odot\textrm{tanh}(c_t)
\end{align}
$$
where $\odot$ represents the element-wise (Hadamard) product. Intuitively, the forget gate $f_t$ allows us to remove or keep values in the cell state when entries are close to 0 or 1, respectively. The input gate activation $i_t$ represents an encoding of which entries in the cell state we may modify and to what degree since these values are between 0 and 1. Together with the cell gate activation $g_t$, which has values from -1 to 1 due to the hyperbolic tangent activation, the element-wise product of $i_t\odot g_t$ allows us to increase or decrease each value in the cell state. Altogether, these mechanisms allow us to retain, forget, or modify information in the cell state may occur from short-term or long-term dependencies in the data. Lastly, the hidden state $h_t$ uses the cell state projected between -1 to 1 by another hyperbolic tangent with one more gate activation $o_t$ to again modify or rescale the information in the cell state. It is important to note that the same weight matrices are used at each time-step.

Finally, multiple LSTM cells may be stacked to form a multi-layer or **stacked LSTM** where the cell state and hidden state of each layer is passed to the next layer at a fixed time-step. Thus, an $L$-layer LSTM with data that has $T$ time-steps will produce $T\times L$ hidden state vectors. For more discussion of LSTMs, [the following GitHub](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) contains an excellent tutorial. PyTorch also has an easy interface for implementing LSTM models using the [nn.LSTM module](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).

The below diagram visualizes the computation of an LSTM cell. In each of the orange sigmoid or hyperbolic tangent layers, the appropriate weight matrices are multiplied into the input or hidden state vectors.

<div>
 <center><img src="lstm-cell.png" width="600"></center>
</div>

In [7]:
import torch
x=torch.tensor([1,23,4])
y=torch.tensor([1,2,3])
z=torch.tensor([6,7,8])
print(z[x==y].numpy())

[6]
